# 2025 DATA·AI 분석 경진대회 - 논문·데이터 추천 에이전트 평가

이 노트북은 LLM 기반 연구 데이터/논문 추천 시스템의 성능을 평가합니다.

**실행 환경**
- GPU: NVIDIA RTX 3080 이상
- CUDA: 11.8+
- Python: 3.10+

## 1. 환경 설정 및 라이브러리 임포트

In [1]:
import sys
import os
import json
import logging
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# 프로젝트 루트 경로
project_root = '/home/infidea/backup-data/paper-reco-agent'
sys.path.insert(0, project_root)

print(f"프로젝트 루트: {project_root}")

# 로깅 설정
os.makedirs(os.path.join(project_root, 'logs'), exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(os.path.join(project_root, 'logs/evaluation.log'))
    ],
    force=True
)

print("✅ 로깅 설정 완료")

# 평가 모듈 임포트
from src.evaluation.metrics import (
    evaluate_recommendations,
    batch_evaluate,
    format_evaluation_report
)
from src.config.settings import settings

print("✅ 평가 모듈 임포트 완료")

프로젝트 루트: /home/infidea/backup-data/paper-reco-agent
✅ 로깅 설정 완료
✅ 평가 모듈 임포트 완료


/home/infidea/backup-data/miniconda3/envs/paper-agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 테스트 데이터셋 로드

In [2]:
# 테스트 데이터셋 로드
testset_path = os.path.join(project_root, 'data', 'test', 'testset_filtered.json')

with open(testset_path, 'r', encoding='utf-8') as f:
    testset = json.load(f)

print(f"✅ 테스트 데이터셋 로드 완료: {testset_path}")
print(f"📊 테스트 카테고리 수: {len(testset.keys())}")

# 모든 테스트 케이스 추출
all_test_cases = []
for category, cases in testset.items():
    for case in cases:
        case['category'] = category
        all_test_cases.append(case)

print(f"📝 총 테스트 케이스: {len(all_test_cases)}개")
print(f"\n첫 번째 테스트 케이스 예시:")
print(json.dumps(all_test_cases[0], indent=2, ensure_ascii=False)[:500] + "...")

✅ 테스트 데이터셋 로드 완료: /home/infidea/backup-data/paper-reco-agent/data/test/testset_filtered.json
📊 테스트 카테고리 수: 2
📝 총 테스트 케이스: 17개

첫 번째 테스트 케이스 예시:
{
  "input_dataset": {
    "svc_id": "c1f03ab868f32371ec97a650bcdf39ac",
    "title": "한반도 기상 관측 데이터 세트",
    "description": "한반도 종관 기상 관측 세스템으로부터의 관측 데이터 세트",
    "keywords": [
      "한반도",
      "기상",
      "센서데이터"
    ]
  },
  "candidate_pool": {
    "papers": [
      {
        "id": "DIKO0014589705",
        "title": "쿠로시오 해류의 변동성과 동아시아 겨울 기후와의 상관성 연구",
        "abstract": "동아시아 겨울 기후를 대표하는 동아시아 겨울 몬순의 변동성은 외부적 다양한 인자들에 의해 영향을 받는다. 북서태평양에서 주요한 흐름인 쿠로시오 해류도 한반도와 주변 바다에 영향을 미치며 동아시아 기후에 영향을 주는...


## 3. 추천 에이전트 초기화

In [3]:
# 추천 에이전트 임포트 및 초기화
from src.agents.recommendation_agent import KoreanResearchRecommendationAgent
from src.config.settings import settings

print("모델 설정:")
print(f"  - 모델명: {settings.MODEL_NAME}")
print(f"  - 임베딩 모델: {settings.EMBEDDING_MODEL}")
print(f"  - 개발 모드: {settings.DEV_MODE}")
print("\n🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)")

agent = KoreanResearchRecommendationAgent()

print("\n✅ 에이전트 초기화 완료")
print(f"모델 정보: {json.dumps(agent.llm_model.get_model_info(), indent=2, ensure_ascii=False)}")

2025-10-15 19:34:28,517 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda:0
2025-10-15 19:34:28,520 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
모델 설정:
  - 모델명: Qwen/Qwen3-14B
  - 임베딩 모델: intfloat/multilingual-e5-large
  - 개발 모드: False

🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)
2025-10-15 19:34:35,150 - src.agents.recommendation_agent - INFO - 🚀 프로덕션 모드로 실행: 실제 LLM 모델 사용
2025-10-15 19:34:35,153 - src.models.llm_model - INFO - 🚀 Qwen 모델 로딩 시작: Qwen/Qwen3-14B
2025-10-15 19:34:35,154 - src.models.llm_model - INFO -    - 디바이스: cuda
2025-10-15 19:34:36,255 - src.models.llm_model - INFO -    - FP16 모드


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 8/8 [01:36<00:00, 12.04s/it]


2025-10-15 19:36:34,117 - src.models.llm_model - INFO - ✅ Qwen 모델 로딩 완료

✅ 에이전트 초기화 완료
모델 정보: {
  "model_name": "Qwen/Qwen3-14B",
  "model_type": "Qwen",
  "device": "cuda",
  "dtype": "float16",
  "max_tokens": 512,
  "temperature": 0.0,
  "parameters": "14.8B",
  "context_length": "32K"
}


## 4. 평가 실행

모든 테스트 케이스에 대해 추천을 생성하고 평가합니다.

In [4]:
# TODO: 추천 개수 설정 (선택사항)
# 최대값: agent의 max_paper_candidates, max_dataset_candidates 값
num_paper_recommendations = 5
num_dataset_recommendations = 5

# agent에서 최대 후보 개수 가져오기
MAX_PAPER_CANDIDATES = agent.max_paper_candidates
MAX_DATASET_CANDIDATES = agent.max_dataset_candidates

print(f"최대 추천 가능 개수: 논문 {MAX_PAPER_CANDIDATES}개, 데이터셋 {MAX_DATASET_CANDIDATES}개")

# 추천 개수 검증
if num_paper_recommendations > MAX_PAPER_CANDIDATES:
    print(f"⚠️  경고: 논문 추천 개수({num_paper_recommendations})가 최대값({MAX_PAPER_CANDIDATES})을 초과합니다.")
    print(f"   자동으로 {MAX_PAPER_CANDIDATES}개로 조정됩니다.")
    num_paper_recommendations = MAX_PAPER_CANDIDATES

if num_dataset_recommendations > MAX_DATASET_CANDIDATES:
    print(f"⚠️  경고: 데이터셋 추천 개수({num_dataset_recommendations})가 최대값({MAX_DATASET_CANDIDATES})을 초과합니다.")
    print(f"   자동으로 {MAX_DATASET_CANDIDATES}개로 조정됩니다.")
    num_dataset_recommendations = MAX_DATASET_CANDIDATES

if num_paper_recommendations < 1:
    print(f"⚠️  경고: 논문 추천 개수는 최소 1개 이상이어야 합니다.")
    num_paper_recommendations = 1

if num_dataset_recommendations < 1:
    print(f"⚠️  경고: 데이터셋 추천 개수는 최소 1개 이상이어야 합니다.")
    num_dataset_recommendations = 1

print(f"✅ 추천 설정: 논문 {num_paper_recommendations}개, 데이터셋 {num_dataset_recommendations}개")

최대 추천 가능 개수: 논문 10개, 데이터셋 10개
✅ 추천 설정: 논문 5개, 데이터셋 5개


In [5]:
# 평가 실행
all_eval_results = []
k_values = [3, 5]

# 추천 원본 결과(recommend_result.json 저장용)
all_recommendations = []

print(f"🔍 {len(all_test_cases)}개 테스트 케이스 평가 시작")
print(f"📏 평가 k 값: {k_values}")
print("=" * 80)

# 모델/임베딩 정보
try:
    _model_info = agent.llm_model.get_model_info()
except Exception:
    _model_info = {}
_model_info_safe = {
    "model_name": _model_info.get("model_name"),
    "model_type": _model_info.get("model_type"),
    "device": _model_info.get("device"),
    "dtype": _model_info.get("dtype"),
    "max_tokens": _model_info.get("max_tokens"),
    "temperature": _model_info.get("temperature"),
    "parameters": _model_info.get("parameters"),
    "context_length": _model_info.get("context_length"),
}

_embedding_info_safe = {
    "embedding_model": getattr(settings, "EMBEDDING_MODEL", None),
    "paper_hybrid_weights": {
        "alpha": getattr(settings, "PAPER_HYBRID_ALPHA", None),
        "beta": getattr(settings, "PAPER_HYBRID_BETA", None),
    },
    "dataset_hybrid_weights": {
        "alpha": getattr(settings, "DATASET_HYBRID_ALPHA", None),
        "beta": getattr(settings, "DATASET_HYBRID_BETA", None),
    },
}

for idx, test_case in enumerate(tqdm(all_test_cases, desc="평가 진행")):
    try:
        dataset_id = test_case['input_dataset']['svc_id']
        category = test_case['category']

        print(f"\n[{idx+1}/{len(all_test_cases)}] {test_case['input_dataset']['title']}")
        print(f"   카테고리: {category}")

        # 추천 생성
        recommendation_result = await agent.recommend(
            dataset_id,
            num_paper_recommendations=num_paper_recommendations,
            num_dataset_recommendations=num_dataset_recommendations
        )

        if 'error' in recommendation_result:
            print(f"   ❌ 오류: {recommendation_result['error']}")
            continue

        # ===== 추천 원본 결과 수집 =====
        # source_dataset
        src_ds = test_case.get("input_dataset", {})
        source_dataset_block = {
            "id": src_ds.get("svc_id"),
            "title": src_ds.get("title"),
            "description": src_ds.get("description"),
            "keywords": src_ds.get("keywords", []),
        }

        # search_result
        search_result = recommendation_result.get("search_result", {})
        search_result_block = {
            "paper_keywords": search_result.get("paper_keywords", []),
            "dataset_keywords": search_result.get("dataset_keywords", []),
            "paper_search_details": search_result.get("paper_search_details", []),
            "dataset_search_details": search_result.get("dataset_search_details", []),
        }

        # paper/dataset recommendations
        paper_reco = recommendation_result.get("paper_recommendations", [])
        dataset_reco = recommendation_result.get("dataset_recommendations", [])

        # candidates_analyzed
        candidates_analyzed = recommendation_result.get(
            "candidates_analyzed",
            (len(recommendation_result.get("paper_candidates", [])) +
             len(recommendation_result.get("dataset_candidates", []))
             if ("paper_candidates" in recommendation_result or "dataset_candidates" in recommendation_result)
             else (len(paper_reco) + len(dataset_reco)))
        )

        # 처리 시간
        processing_time_ms = recommendation_result.get("processing_time_ms", None)

        # 하나의 케이스 레코드 구성
        reco_record = {
            "source_dataset": source_dataset_block,
            "search_result": search_result_block,
            "paper_recommendations": paper_reco,
            "dataset_recommendations": dataset_reco,
            "processing_time_ms": processing_time_ms,
            "candidates_analyzed": candidates_analyzed,
            "model_info": _model_info_safe,
            "embedding_model_info": _embedding_info_safe,
        }
        all_recommendations.append(reco_record)
        # ==============================================

        # 평가 형식으로 변환
        predictions = []
        for paper in recommendation_result.get('paper_recommendations', []):
            predictions.append({
                'type': 'paper',
                'id': paper['id'],
                'rank': paper.get('rank'),
                'score': paper.get('score')
            })

        for dataset in recommendation_result.get('dataset_recommendations', []):
            predictions.append({
                'type': 'dataset',
                'id': dataset['id'],
                'rank': dataset.get('rank'),
                'score': dataset.get('score')
            })

        # 평가
        ground_truth = test_case['candidate_pool']
        eval_result = evaluate_recommendations(predictions, ground_truth, k_values)

        # 메타데이터 추가
        eval_result['test_case_index'] = idx
        eval_result['dataset_id'] = dataset_id
        eval_result['category'] = category
        eval_result['input_title'] = test_case['input_dataset']['title']
        eval_result['processing_time_ms'] = processing_time_ms if processing_time_ms is not None else recommendation_result.get('processing_time_ms', 0)
        # 선택: 추천 개수 기록(분석 편의)
        eval_result['num_predicted_papers'] = len([p for p in predictions if p['type'] == 'paper'])
        eval_result['num_predicted_datasets'] = len([p for p in predictions if p['type'] == 'dataset'])
        all_eval_results.append(eval_result)

        # 결과 요약 출력 (k_values 반영)
        summary_chunks = []
        for k in k_values:
            ndcg = eval_result.get(f"overall_ndcg@{k}", 0)
            mrr = eval_result.get(f"overall_mrr@{k}", 0)
            recall = eval_result.get(f"overall_recall@{k}", 0)
            summary_chunks.append(
                f"nDCG@{k}={ndcg:.4f}, MRR@{k}={mrr:.4f}, Recall@{k}={recall:.4f}"
            )
        print("   ✅ " + " | ".join(summary_chunks))

    except Exception as e:
        print(f"   ❌ 예외 발생: {e}")
        logging.error(f"테스트 케이스 {idx} 평가 실패: {e}", exc_info=True)
        continue

print("\n" + "=" * 80)
print(f"✅ 평가 완료: {len(all_eval_results)}/{len(all_test_cases)}개 케이스 성공")

🔍 17개 테스트 케이스 평가 시작
📏 평가 k 값: [3, 5]


평가 진행:   0%|          | 0/17 [00:00<?, ?it/s]


[1/17] 한반도 기상 관측 데이터 세트
   카테고리: 국내
2025-10-15 19:36:34,142 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID c1f03ab868f32371ec97a650bcdf39ac
2025-10-15 19:36:34,257 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/c1f03ab868f32371ec97a650bcdf39ac?key=4936BC43D48603524DEDA2E2D56D6B46 "HTTP/1.1 200 200"
2025-10-15 19:36:34,260 - src.clients.dataon_client - INFO - Successfully retrieved metadata for dataset c1f03ab868f32371ec97a650bcdf39ac
2025-10-15 19:36:34,261 - src.clients.dataon_client - INFO - API Response:
{
  "response": {
    "elapsed time": "35 ms",
    "status": "200",
    "message": "OK",
    "total count": "1",
    "type": "json"
  },
  "records": {
    "svc_id": "c1f03ab868f32371ec97a650bcdf39ac",
    "ctlg_type": "02",
    "dataset_type": "01",
    "ctlg_type_pc": "dataset",
    "dataset_type_pc": "국내",
    "dataset_pub_dt_pc": "2025",
    "dataset_access_type_pc": "공개",
    "file_yn_pc": "직접다운로드",
    "dataset_cc_license


Batches: 100%|██████████| 1/1 [00:00<00:00, 14.61it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 100.76it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 68.04it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 85.35it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 136.13it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 90.88it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 137.15it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 87.36it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 123.31it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 64.01it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 102.23it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 79.59it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 133.64it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 131.30it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 111.92it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 102.58it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 134.42it/s]

Batches: 100%|██████████| 1/1 [00:00<

2025-10-15 19:36:39,940 - src.agents.recommendation_agent - INFO - 상위 25개 논문, 18개 데이터셋 순위 결정 완료


...

   ✅ nDCG@3=0.0000, MRR@3=0.0000, Recall@3=0.0000 | nDCG@5=0.0000, MRR@5=0.0000, Recall@5=0.0000

[17/17] Police recorded crime trends in Victoria during the COVID-19 pandemic
   카테고리: 해외
2025-10-15 19:47:16,198 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID b89d0c5d5891c5c80cd7a565ce8a3dc6
2025-10-15 19:47:16,301 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/b89d0c5d5891c5c80cd7a565ce8a3dc6?key=4936BC43D48603524DEDA2E2D56D6B46 "HTTP/1.1 200 200"
2025-10-15 19:47:16,304 - src.clients.dataon_client - INFO - Successfully retrieved metadata for dataset b89d0c5d5891c5c80cd7a565ce8a3dc6
2025-10-15 19:47:16,305 - src.clients.dataon_client - INFO - API Response:
{
  "response": {
    "elapsed time": "30 ms",
    "status": "200",
    "message": "OK",
    "total count": "1",
    "type": "json"
  },
  "records": {
    "svc_id": "b89d0c5d5891c5c80cd7a565ce8a3dc6",
    "ctlg_type": "02",
    "dataset_type": "02",
    "ctlg_type_pc": "dataset"


Batches: 100%|██████████| 1/1 [00:00<00:00, 83.42it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 61.09it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 93.87it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 79.30it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 94.07it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 81.09it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 91.87it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 80.93it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 93.64it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 61.35it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 94.35it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 80.09it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 92.00it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 74.81it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 91.71it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 161.77it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 95.58it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 63

2025-10-15 19:47:20,899 - src.agents.recommendation_agent - INFO - 상위 25개 논문, 5개 데이터셋 순위 결정 완료
2025-10-15 19:47:20,900 - src.agents.recommendation_agent - INFO - LLM paper 추천 생성 시도 1/2
2025-10-15 19:47:20,901 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-15 19:47:20,901 - src.agents.recommendation_agent - INFO - LLM에게 전송하는 paper 프롬프트:
2025-10-15 19:47:20,902 - src.agents.recommendation_agent - INFO - # Task: Re-rank and Select Top 5 paper Recommendations
You are a research recommendation expert. Re-rank the candidates and select the top 5 most relevant items.
## Source Dataset:
Title: Police recorded crime trends in Victoria during the COVID-19 pandemic
Description: The Crime Statistics Agency (CSA) has released a research paper entitled Police-recorded crime trends in Victoria during the COVID-19 pandemic. The paper examines trends in criminal offences and family violence incidents to 30 June 2020, f

평가 진행: 100%|██████████| 17/17 [11:23<00:00, 40.20s/it]

   ✅ nDCG@3=0.0000, MRR@3=0.0000, Recall@3=0.0000 | nDCG@5=0.0000, MRR@5=0.0000, Recall@5=0.0000

✅ 평가 완료: 17/17개 케이스 성공


## 5. 평가 결과 분석 및 저장

### 5.1 평가 결과 저장 디렉토리 설정

In [10]:
# 타임스탬프 기반 결과 디렉토리 생성
timestamp = datetime.now().strftime('%Y%m%d%H%M')
eval_output_dir = os.path.join(project_root, 'figures', 'evaluation_results', timestamp)
os.makedirs(eval_output_dir, exist_ok=True)

print(f"📂 평가 결과 저장 경로: {eval_output_dir}")

# 추천 원본 결과 저장 
try:
    recommend_path = os.path.join(eval_output_dir, 'recommend_result.json')
    with open(recommend_path, 'w', encoding='utf-8') as f:
        json.dump(all_recommendations, f, indent=2, ensure_ascii=False)
    print(f"💾 recommend_result.json 저장 완료 → {recommend_path}")
except Exception as e:
    print(f"⚠️ recommend_result.json 저장 실패: {e}")

📂 평가 결과 저장 경로: /home/infidea/backup-data/paper-reco-agent/figures/evaluation_results/202510151948
💾 recommend_result.json 저장 완료 → /home/infidea/backup-data/paper-reco-agent/figures/evaluation_results/202510151948/recommend_result.json


### 5.2 전체 메트릭 계산

In [11]:
if not all_eval_results:
    print("⚠️  평가 결과가 없습니다")
else:
    # DataFrame 변환
    df_results = pd.DataFrame(all_eval_results)

    print("\n" + "=" * 80)
    print("📊 평균 평가 메트릭")
    print("=" * 80)

    # 메트릭 요약 계산
    metrics_summary = {}

    for k in k_values:
        print(f"\n📏 k={k} 결과:")
        print("-" * 80)

        k_metrics = {}

        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            print(f"\n  {type_label}:")

            type_metrics = {}
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@{k}'
                if col_name in df_results.columns:
                    mean_val = df_results[col_name].mean()
                    std_val = df_results[col_name].std()
                    type_metrics[metric] = {'mean': mean_val, 'std': std_val}
                    print(f"    • {metric.upper():10s}: {mean_val:.4f} ± {std_val:.4f}")

            k_metrics[metric_type] = type_metrics

        metrics_summary[f'k{k}'] = k_metrics

    # 통계
    print("\n" + "=" * 80)
    print("📈 기본 통계")
    print("=" * 80)
    print(f"  평균 추천 논문 수: {df_results['num_predicted_papers'].mean():.2f}개")
    print(f"  평균 추천 데이터셋 수: {df_results['num_predicted_datasets'].mean():.2f}개")
    print(f"  평균 관련 논문 수: {df_results['num_relevant_papers'].mean():.2f}개")
    print(f"  평균 관련 데이터셋 수: {df_results['num_relevant_datasets'].mean():.2f}개")
    print(f"  평균 처리 시간: {df_results['processing_time_ms'].mean():.0f} ms")


📊 평균 평가 메트릭

📏 k=3 결과:
--------------------------------------------------------------------------------

  전체:
    • NDCG      : 0.0588 ± 0.2425
    • MRR       : 0.0588 ± 0.2425
    • RECALL    : 0.0042 ± 0.0173
    • PRECISION : 0.0196 ± 0.0808

  논문:
    • NDCG      : 0.0588 ± 0.2425
    • MRR       : 0.0588 ± 0.2425
    • RECALL    : 0.0084 ± 0.0346
    • PRECISION : 0.0196 ± 0.0808

  데이터셋:
    • NDCG      : 0.3283 ± 0.4656
    • MRR       : 0.3235 ± 0.4657
    • RECALL    : 0.1375 ± 0.2294
    • PRECISION : 0.2647 ± 0.4042

📏 k=5 결과:
--------------------------------------------------------------------------------

  전체:
    • NDCG      : 0.0588 ± 0.2425
    • MRR       : 0.0588 ± 0.2425
    • RECALL    : 0.0042 ± 0.0173
    • PRECISION : 0.0118 ± 0.0485

  논문:
    • NDCG      : 0.0588 ± 0.2425
    • MRR       : 0.0588 ± 0.2425
    • RECALL    : 0.0084 ± 0.0346
    • PRECISION : 0.0118 ± 0.0485

  데이터셋:
    • NDCG      : 0.3252 ± 0.4610
    • MRR       : 0.3235 ± 0.4657
    • REC

### 5.3 카테고리별 분석

In [12]:
if all_eval_results:
    print("\n" + "=" * 80)
    print("📊 카테고리별 메트릭 (k=5)")
    print("=" * 80)

    category_metrics = {}

    for category in sorted(df_results['category'].unique()):
        category_data = df_results[df_results['category'] == category]

        print(f"\n📁 {category} (n={len(category_data)}):")
        print("-" * 80)

        cat_metrics = {}
        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            print(f"  {type_label}:")

            type_metrics = {}
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@5'
                if col_name in category_data.columns:
                    mean_val = category_data[col_name].mean()
                    type_metrics[metric] = mean_val
                    print(f"    • {metric.upper():10s}: {mean_val:.4f}")

            cat_metrics[metric_type] = type_metrics

        category_metrics[category] = cat_metrics


📊 카테고리별 메트릭 (k=5)

📁 국내 (n=15):
--------------------------------------------------------------------------------
  전체:
    • NDCG      : 0.0667
    • MRR       : 0.0667
    • RECALL    : 0.0048
    • PRECISION : 0.0133
  논문:
    • NDCG      : 0.0667
    • MRR       : 0.0667
    • RECALL    : 0.0095
    • PRECISION : 0.0133
  데이터셋:
    • NDCG      : 0.3685
    • MRR       : 0.3667
    • RECALL    : 0.1780
    • PRECISION : 0.2267

📁 해외 (n=2):
--------------------------------------------------------------------------------
  전체:
    • NDCG      : 0.0000
    • MRR       : 0.0000
    • RECALL    : 0.0000
    • PRECISION : 0.0000
  논문:
    • NDCG      : 0.0000
    • MRR       : 0.0000
    • RECALL    : 0.0000
    • PRECISION : 0.0000
  데이터셋:
    • NDCG      : 0.0000
    • MRR       : 0.0000
    • RECALL    : 0.0000
    • PRECISION : 0.0000


### 5.4 최종 결과 저장

**📁 저장되는 파일:**
1. **`EVALUATION_SUMMARY.txt`** ⭐ - 가장 먼저 봐야 할 요약 리포트
2. **`detailed_results.csv`** - 엑셀에서 열어볼 수 있는 상세 데이터
3. **`config.json`** - 실험 재현을 위한 설정 정보
4. **`metrics.json`** - 프로그램으로 읽을 수 있는 메트릭 데이터

In [13]:
if all_eval_results:
    print("\n💾 결과 저장 중...\n")

    # 1. 요약 리포트 생성 (가장 중요!)
    report_lines = []
    report_lines.append("=" * 80)
    report_lines.append("평가 요약 리포트")
    report_lines.append("=" * 80)
    report_lines.append("")
    report_lines.append(f"📅 평가 시각: {timestamp}")
    report_lines.append(f"🤖 LLM 모델: {settings.MODEL_NAME}")
    report_lines.append(f"🔍 임베딩 모델: {settings.EMBEDDING_MODEL}")
    report_lines.append(f"📊 테스트 케이스: {len(all_eval_results)}/{len(all_test_cases)} 성공")
    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("주요 성능 지표 (k=5)")
    report_lines.append("=" * 80)
    report_lines.append("")

    # k=5 전체 메트릭
    for metric in ['ndcg', 'mrr', 'recall', 'precision']:
        col_name = f'overall_{metric}@5'
        if col_name in df_results.columns:
            mean_val = df_results[col_name].mean()
            report_lines.append(f"  {metric.upper():10s} @5: {mean_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("타입별 성능 (k=5)")
    report_lines.append("=" * 80)

    for metric_type in ['paper', 'dataset']:
        type_label = {'paper': '논문', 'dataset': '데이터셋'}[metric_type]
        report_lines.append("")
        report_lines.append(f"[{type_label}]")
        for metric in ['ndcg', 'mrr', 'recall', 'precision']:
            col_name = f'{metric_type}_{metric}@5'
            if col_name in df_results.columns:
                mean_val = df_results[col_name].mean()
                report_lines.append(f"  {metric.upper():10s}: {mean_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("카테고리별 nDCG@5")
    report_lines.append("=" * 80)
    report_lines.append("")

    for category in sorted(df_results['category'].unique()):
        category_data = df_results[df_results['category'] == category]
        mean_val = category_data['overall_ndcg@5'].mean()
        report_lines.append(f"  {category:30s}: {mean_val:.4f} (n={len(category_data)})")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("전체 메트릭 (모든 k 값)")
    report_lines.append("=" * 80)

    for k in k_values:
        report_lines.append("")
        report_lines.append(f"k={k}:")
        report_lines.append("-" * 40)
        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            report_lines.append(f"  [{type_label}]")
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@{k}'
                if col_name in df_results.columns:
                    mean_val = df_results[col_name].mean()
                    std_val = df_results[col_name].std()
                    report_lines.append(f"    {metric.upper():10s}: {mean_val:.4f} ± {std_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)

    # 요약 리포트 저장
    summary_path = os.path.join(eval_output_dir, 'EVALUATION_SUMMARY.txt')
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_lines))
    print(f"✅ 1. 요약 리포트 저장: EVALUATION_SUMMARY.txt")

    # 2. 상세 CSV 저장
    csv_path = os.path.join(eval_output_dir, 'detailed_results.csv')
    df_results.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"✅ 2. 상세 결과 CSV 저장: detailed_results.csv")

    # 3. 메트릭 JSON 저장
    all_metrics = {
        'summary': metrics_summary,
        'by_category': category_metrics,
        'individual_results': all_eval_results
    }

    metrics_path = os.path.join(eval_output_dir, 'metrics.json')
    with open(metrics_path, 'w', encoding='utf-8') as f:
        json.dump(all_metrics, f, indent=2, ensure_ascii=False)
    print(f"✅ 3. 메트릭 JSON 저장: metrics.json")

    print(f"\n" + "=" * 80)
    print(f"📂 모든 결과가 저장되었습니다:")
    print(f"   {eval_output_dir}")
    print(f"" + "=" * 80)
    print(f"\n⭐ 결과 확인: EVALUATION_SUMMARY.txt 파일을 먼저 읽어보세요!")

    # 요약 리포트 미리보기
    print(f"\n" + "=" * 80)
    print("📄 요약 리포트 미리보기:")
    print("=" * 80)
    print('\n'.join(report_lines[:50]))  # 처음 50줄만 표시
    if len(report_lines) > 50:
        print("\n... (전체 내용은 EVALUATION_SUMMARY.txt 참고) ...")
else:
    print("⚠️  저장할 결과가 없습니다")


💾 결과 저장 중...

✅ 1. 요약 리포트 저장: EVALUATION_SUMMARY.txt
✅ 2. 상세 결과 CSV 저장: detailed_results.csv
✅ 3. 메트릭 JSON 저장: metrics.json

📂 모든 결과가 저장되었습니다:
   /home/infidea/backup-data/paper-reco-agent/figures/evaluation_results/202510151948

⭐ 결과 확인: EVALUATION_SUMMARY.txt 파일을 먼저 읽어보세요!

📄 요약 리포트 미리보기:
평가 요약 리포트

📅 평가 시각: 202510151948
🤖 LLM 모델: Qwen/Qwen3-14B
🔍 임베딩 모델: intfloat/multilingual-e5-large
📊 테스트 케이스: 17/17 성공

주요 성능 지표 (k=5)

  NDCG       @5: 0.0588
  MRR        @5: 0.0588
  RECALL     @5: 0.0042
  PRECISION  @5: 0.0118

타입별 성능 (k=5)

[논문]
  NDCG      : 0.0588
  MRR       : 0.0588
  RECALL    : 0.0084
  PRECISION : 0.0118

[데이터셋]
  NDCG      : 0.3252
  MRR       : 0.3235
  RECALL    : 0.1571
  PRECISION : 0.2000

카테고리별 nDCG@5

  국내                            : 0.0667 (n=15)
  해외                            : 0.0000 (n=2)

전체 메트릭 (모든 k 값)

k=3:
----------------------------------------
  [전체]
    NDCG      : 0.0588 ± 0.2425
    MRR       : 0.0588 ± 0.2425

... (전체 내용은 EVALUATION_SUMMARY.tx

## 7. 평가 완료

**결과 파일 안내**
- 평가 결과는 `figures/evaluation_results/{timestamp}/` 디렉토리에 저장됩니다.

**파일 읽는 순서:**
1. `EVALUATION_SUMMARY.txt` ⭐⭐⭐
   - 가장 먼저 읽어야 할 파일
   - 주요 성능 지표와 카테고리별 결과 요약
   - 텍스트 에디터로 바로 열어볼 수 있음

2. `detailed_results.csv`
   - 엑셀이나 스프레드시트로 열어서 상세 분석
   - 각 테스트 케이스별 메트릭 포함
   - 필터링, 정렬, 차트 생성 가능

3. `recommend_result.json`
   - 평가 데이터 추천 결과

4. `metrics.json`
   - 프로그램으로 읽을 수 있는 메트릭 데이터
   - 자동화된 분석이나 비교에 사용

**주요 메트릭 설명**
- nDCG@k: 순위를 고려한 추천 품질 (0~1, 높을수록 좋음)
- MRR@k: 첫 번째 관련 항목의 순위 (0~1, 높을수록 좋음)
- Recall@k: 관련 항목 중 추천된 비율 (0~1, 높을수록 좋음)
- Precision@k: 추천 항목 중 관련된 비율 (0~1, 높을수록 좋음)